In [14]:
import env
import utilities as util
import pandas as pd

from wrangle_zillow import wrangle_zillow_df

# Clustering - Data Wrangling

## 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

* Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
* Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
* Only include properties that include a latitude and longitude value.

In [2]:
zillow_query = """
SELECT *
	FROM predictions_2017 pred
		JOIN (
			SELECT parcelid, max(transactiondate) AS latest_transaction
				FROM predictions_2017
				GROUP BY parcelid) trans
			ON pred.parcelid = trans.parcelid AND pred.transactiondate = trans.latest_transaction
		JOIN properties_2017 prop ON pred.parcelid = prop.parcelid
		LEFT JOIN airconditioningtype ac ON prop.airconditioningtypeid = ac.airconditioningtypeid
		LEFT JOIN architecturalstyletype ar ON prop.architecturalstyletypeid = ar.architecturalstyletypeid
		LEFT JOIN buildingclasstype bc ON prop.buildingclasstypeid = bc.buildingclasstypeid
		LEFT JOIN heatingorsystemtype ht ON prop.heatingorsystemtypeid = ht.heatingorsystemtypeid
		LEFT JOIN propertylandusetype pl ON prop.propertylandusetypeid = pl.propertylandusetypeid
		LEFT JOIN storytype st ON prop.storytypeid = st.storytypeid
		LEFT JOIN typeconstructiontype ct ON prop.typeconstructiontypeid = ct.typeconstructiontypeid
	WHERE latitude IS NOT NULL
		AND longitude IS NOT NULL
"""

In [3]:
zillow_df = util.generate_df("zillow_clustering.csv", zillow_query, util.generate_db_url(env.user, env.password, env.host, "zillow"))

## 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [14]:
zillow_df.head(5)

,id,parcelid,logerror,transactiondate,parcelid.1,latest_transaction,id.1,parcelid.2,airconditioningtypeid,architecturalstyletypeid,...,buildingclasstypeid.1,buildingclassdesc,heatingorsystemtypeid.1,heatingorsystemdesc,propertylandusetypeid.1,propertylandusedesc,storytypeid.1,storydesc,typeconstructiontypeid.1,typeconstructiondesc
0,0,14297519,0.025595,2017-01-01,14297519,2017-01-01,1727539,14297519,NaN,NaN,...,NaN,NaN,NaN,NaN,261,Single Family Residential,NaN,NaN,NaN,NaN
1,1,17052889,0.055619,2017-01-01,17052889,2017-01-01,1387261,17052889,NaN,NaN,...,NaN,NaN,NaN,NaN,261,Single Family Residential,NaN,NaN,NaN,NaN
2,2,14186244,0.005383,2017-01-01,14186244,2017-01-01,11677,14186244,NaN,NaN,...,NaN,NaN,NaN,NaN,261,Single Family Residential,NaN,NaN,NaN,NaN
3,3,12177905,-0.103410,2017-01-01,12177905,2017-01-01,2288172,12177905,NaN,NaN,...,NaN,NaN,2.0,Central,261,Single Family Residential,NaN,NaN,NaN,NaN
4,4,10887214,0.006940,2017-01-01,10887214,2017-01-01,1970746,10887214,1.0,NaN,...,NaN,NaN,2.0,Central,266,Condominium,NaN,NaN,NaN,NaN


In [6]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77381 non-null  int64  
 1   parcelid                      77381 non-null  int64  
 2   logerror                      77381 non-null  float64
 3   transactiondate               77381 non-null  object 
 4   parcelid.1                    77381 non-null  int64  
 5   latest_transaction            77381 non-null  object 
 6   id.1                          77381 non-null  int64  
 7   parcelid.2                    77381 non-null  int64  
 8   airconditioningtypeid         24953 non-null  float64
 9   architecturalstyletypeid      206 non-null    float64
 10  basementsqft                  50 non-null     float64
 11  bathroomcnt                   77381 non-null  float64
 12  bedroomcnt                    77381 non-null  float64
 13  b

In [7]:
zillow_df.shape

(77381, 79)

In [10]:
zillow_df.describe()

,id,parcelid,logerror,parcelid.1,id.1,parcelid.2,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,taxamount,taxdelinquencyyear,censustractandblock,airconditioningtypeid.1,architecturalstyletypeid.1,buildingclasstypeid.1,heatingorsystemtypeid.1,propertylandusetypeid.1,storytypeid.1,typeconstructiontypeid.1
count,77381.000000,7.738100e+04,77381.000000,7.738100e+04,7.738100e+04,7.738100e+04,24953.000000,206.000000,50.000000,77381.000000,...,77376.000000,2886.000000,7.713700e+04,24953.000000,206.000000,15.000000,49440.000000,77381.000000,50.0,222.000000
mean,38848.692754,1.300715e+07,0.016625,1.300715e+07,1.495139e+06,1.300715e+07,1.813289,7.388350,679.720000,2.299130,...,5995.532346,14.087318,6.049673e+13,1.813289,7.388350,3.933333,3.920510,261.826340,7.0,6.040541
std,22402.429065,3.481346e+06,0.170191,3.481346e+06,8.609071e+05,3.481346e+06,2.967894,2.734542,689.703546,0.996651,...,7622.844153,2.185663,1.535242e+12,2.967894,2.734542,0.258199,3.592779,5.141199,0.0,0.557285
min,0.000000,1.071186e+07,-4.655420,1.071186e+07,3.490000e+02,1.071186e+07,1.000000,2.000000,38.000000,0.000000,...,19.920000,3.000000,6.037101e+13,1.000000,2.000000,3.000000,1.000000,31.000000,7.0,4.000000
25%,19461.000000,1.153830e+07,-0.024377,1.153830e+07,7.520700e+05,1.153830e+07,1.000000,7.000000,273.000000,2.000000,...,2715.632500,14.000000,6.037311e+13,1.000000,7.000000,4.000000,2.000000,261.000000,7.0,6.000000
50%,38870.000000,1.253157e+07,0.006627,1.253157e+07,1.497932e+06,1.253157e+07,1.000000,7.000000,515.000000,2.000000,...,4450.695000,15.000000,6.037603e+13,1.000000,7.000000,4.000000,2.000000,261.000000,7.0,6.000000
75%,58253.000000,1.421183e+07,0.039203,1.421183e+07,2.240535e+06,1.421183e+07,1.000000,7.000000,796.500000,3.000000,...,6927.790000,15.000000,6.059042e+13,1.000000,7.000000,4.000000,7.000000,266.000000,7.0,6.000000
max,77613.000000,1.676893e+08,5.262999,1.676893e+08,2.982274e+06,1.676893e+08,13.000000,21.000000,3560.000000,18.000000,...,586639.300000,99.000000,4.830301e+14,13.000000,21.000000,4.000000,24.000000,275.000000,7.0,13.000000


## 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [4]:
def show_missing_value_stats_by_col(df):
    cols = df.columns
    rows = len(df)
    result = pd.DataFrame(index=cols, columns=['num_rows_missing', 'pct_rows_missing'])
    pd.set_option('max_rows', rows)
    
    result['num_rows_missing'] = df.isnull().sum()
    result['pct_rows_missing'] = round(df.isnull().sum() / rows, 6)
    
    return result

In [37]:
show_missing_value_stats_by_col(zillow_df)

,num_rows_missing,pct_rows_missing
id,0,0.000000
parcelid,0,0.000000
logerror,0,0.000000
transactiondate,0,0.000000
parcelid.1,0,0.000000
latest_transaction,0,0.000000
id.1,0,0.000000
parcelid.2,0,0.000000
airconditioningtypeid,52428,0.677531
architecturalstyletypeid,77175,0.997338


There are several columns with over 90% of values missing which should be dropped. Any column with less than 50% values missing will be imputed. If the missing values percent is less than 10%, the column should be investigated for errors.

## 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [5]:
def show_missing_value_stats_by_row(df):
    total_cols = df.shape[1]
    total_rows = df.shape[0]
    result = pd.DataFrame(df.isnull().sum(axis=1).value_counts(), columns=['num_rows'])
    pd.set_option('max_rows', total_rows)
    
    result = result.reset_index()
    result = result.rename(columns={'index' : 'num_cols_missing'})
    result['pct_cols_missing'] = result['num_cols_missing'] / total_cols
    result = result.set_index('num_cols_missing')
    result = result.sort_values('num_cols_missing', ascending=True)
    
    return result

In [36]:
show_missing_value_stats_by_row(zillow_df)

,num_rows,pct_cols_missing
num_cols_missing,,
25,2,0.316456
26,11,0.329114
27,21,0.341772
28,39,0.354430
29,43,0.367089
30,36,0.379747
31,230,0.392405
32,178,0.405063
33,4827,0.417722


A large number of rows are missing values for 33-44 columns. We will need to drop these columns or impute them in order to maintain enough data points.

# Prepare

## 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [6]:
single_property_landusetypes = (zillow_df.propertylandusetypeid == 261) | (zillow_df.propertylandusetypeid == 279) | (zillow_df.propertylandusetypeid == 262) | (zillow_df.propertylandusetypeid == 263) | (zillow_df.propertylandusetypeid == 264) | (zillow_df.propertylandusetypeid == 266) | (zillow_df.propertylandusetypeid == 276)

single_unit_properties_df = zillow_df[(zillow_df.unitcnt == 1) | single_property_landusetypes]

In [7]:
single_unit_properties_df.shape

(73700, 79)

## 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

* The input:
    * A dataframe
    * A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    * A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
* The output:
    * The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
* hint:
    * Look up the dropna documentation.
    * You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    * Make use of inplace, i.e. inplace=True/False.

In [8]:
def handle_missing_values(df, col_thresh, row_thresh):
    req_col = int(round(col_thresh * len(df.index), 0))
    req_row = int(round(row_thresh * len(df.columns), 0))
    
    df = df.dropna(axis=1, thresh=req_col)
    df = df.dropna(axis=0, thresh=req_row)
    
    return df

## 3. Decide how to handle the remaining missing values:

* Fill with constant value.
* Impute with mean, median, mode.
* Drop row/column

In [10]:
prepped_zillow = handle_missing_values(single_unit_properties_df, .5, .5)

In [11]:
show_missing_value_stats_by_col(prepped_zillow)

,num_rows_missing,pct_rows_missing
id,0,0.0
parcelid,0,0.0
logerror,0,0.0
transactiondate,0,0.0
parcelid.1,0,0.0
latest_transaction,0,0.0
id.1,0,0.0
parcelid.2,0,0.0
bathroomcnt,0,0.0
bedroomcnt,0,0.0


In [12]:
show_missing_value_stats_by_row(prepped_zillow)

,num_rows,pct_cols_missing
num_cols_missing,,
0,43778,0.0


,id,parcelid,logerror,transactiondate,parcelid.1,latest_transaction,id.1,parcelid.2,bathroomcnt,bedroomcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemtypeid.1,heatingorsystemdesc,propertylandusetypeid.1,propertylandusedesc
5114,5146,11518287,0.035537,2017-01-24,11518287,2017-01-24,2539056,11518287,2.0,2.0,...,72913.0,150969.0,2016.0,78056.0,1827.62,6.037236e+13,7.0,Floor/Wall,266,Condominium
44052,44246,13049110,0.010928,2017-06-07,13049110,2017-06-07,412868,13049110,1.0,2.0,...,54804.0,242234.0,2016.0,187430.0,2932.57,6.037402e+13,2.0,Central,269,Planned Unit Development
43410,43600,11515551,-0.022381,2017-06-06,11515551,2017-06-06,958486,11515551,3.0,2.0,...,189308.0,378616.0,2016.0,189308.0,4246.62,6.037621e+13,2.0,Central,266,Condominium
20785,20906,11544001,0.028108,2017-03-28,11544001,2017-03-28,809493,11544001,2.0,3.0,...,32074.0,76373.0,2016.0,44299.0,1015.77,6.037272e+13,7.0,Floor/Wall,261,Single Family Residential
19247,19362,11314617,0.049255,2017-03-22,11314617,2017-03-22,2896227,11314617,2.0,4.0,...,50472.0,63088.0,2016.0,12616.0,1549.23,6.037901e+13,2.0,Central,261,Single Family Residential
31837,31999,10905659,-0.201039,2017-05-02,10905659,2017-05-02,1732265,10905659,2.0,2.0,...,49065.0,102269.0,2016.0,53204.0,1301.95,6.037143e+13,2.0,Central,261,Single Family Residential
6619,6658,12863978,0.057541,2017-01-31,12863978,2017-01-31,203041,12863978,3.0,4.0,...,179522.0,276494.0,2016.0,96972.0,3672.45,6.037402e+13,2.0,Central,261,Single Family Residential
13111,13192,12942835,-0.005504,2017-02-28,12942835,2017-02-28,2216801,12942835,2.0,3.0,...,50713.0,61663.0,2016.0,10950.0,906.36,6.037407e+13,2.0,Central,261,Single Family Residential
68814,69043,12098494,-0.067088,2017-08-22,12098494,2017-08-22,2585867,12098494,5.0,5.0,...,1147520.0,2294391.0,2016.0,1146871.0,26122.85,6.037464e+13,2.0,Central,261,Single Family Residential
65157,65381,11276931,-0.008474,2017-08-10,11276931,2017-08-10,2329606,11276931,2.0,4.0,...,95237.0,119044.0,2016.0,23807.0,2214.33,6.037901e+13,2.0,Central,261,Single Family Residential


In [15]:
prepped_zillow_df = wrangle_zillow_df()
prepped_zillow_df

,id,parcelid,logerror,transactiondate,parcelid.1,latest_transaction,id.1,parcelid.2,bathroomcnt,bedroomcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemtypeid.1,heatingorsystemdesc,propertylandusetypeid.1,propertylandusedesc
3,3,12177905,-0.103410,2017-01-01,12177905,2017-01-01,2288172,12177905,3.0,4.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,2.0,Central,261,Single Family Residential
4,4,10887214,0.006940,2017-01-01,10887214,2017-01-01,1970746,10887214,3.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,2.0,Central,266,Condominium
6,6,12095076,-0.001011,2017-01-01,12095076,2017-01-01,781532,12095076,3.0,4.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,2.0,Central,261,Single Family Residential
8,8,12790562,-0.040966,2017-01-02,12790562,2017-01-02,1246926,12790562,3.0,4.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,2.0,Central,261,Single Family Residential
9,9,11542646,-0.036763,2017-01-02,11542646,2017-01-02,1585097,11542646,3.0,2.0,...,151303.0,371361.0,2016.0,220058.0,4557.32,6.037275e+13,2.0,Central,266,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,77608,10833991,-0.002245,2017-09-20,10833991,2017-09-20,2864704,10833991,3.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,2.0,Central,266,Condominium
77376,77609,11000655,0.020615,2017-09-20,11000655,2017-09-20,673515,11000655,2.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,2.0,Central,261,Single Family Residential
77378,77611,12773139,0.037129,2017-09-21,12773139,2017-09-21,1843709,12773139,1.0,3.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,2.0,Central,261,Single Family Residential
77379,77612,12826780,0.007204,2017-09-25,12826780,2017-09-25,1187175,12826780,2.0,3.0,...,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,2.0,Central,261,Single Family Residential


In [16]:
show_missing_value_stats_by_col(prepped_zillow_df)

,num_rows_missing,pct_rows_missing
id,0,0.0
parcelid,0,0.0
logerror,0,0.0
transactiondate,0,0.0
parcelid.1,0,0.0
latest_transaction,0,0.0
id.1,0,0.0
parcelid.2,0,0.0
bathroomcnt,0,0.0
bedroomcnt,0,0.0


In [17]:
show_missing_value_stats_by_row(prepped_zillow_df)

,num_rows,pct_cols_missing
num_cols_missing,,
0,43779,0.0
